## Semantic Kernel: Ramp-Up based on SK's Documentation

To get the latest version of SK Python package, use:

``` bash
pip install --upgrade semantic-kernel
```

## 📒 Notebook 2: AI Services

### 🪜 Step 1: Configure environment

In [1]:
# Import required packages
import os
from IPython.display import Image, display

from semantic_kernel import Kernel
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings

In [2]:
# Set Azure OpenAI backend variables
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")
AOAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

### 🪜 Step 2: Add AI service

In [3]:
# Initialise kernel
kernel = Kernel()

In [4]:
# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    deployment_name = AOAI_DEPLOYMENT,
    endpoint = AOAI_ENDPOINT,
    api_version = AOAI_API_VERSION,
    service_id = "azure_openai_chat",
)

kernel.add_service(chat_completion)

### 🪜 Step 3: Retrieve AI service

In [5]:
# Retrieve the chat completion service by type
chat_completion_service = kernel.get_service(type=ChatCompletionClientBase)
chat_completion_service.ai_model_type

<OpenAIModelTypes.CHAT: 'chat'>

In [6]:
# Retrieve the chat completion service by id
chat_completion_service = kernel.get_service(service_id="azure_openai_chat")
chat_completion_service.ai_model_type

<OpenAIModelTypes.CHAT: 'chat'>

### 🪜 Step 4: Direct use of Chat Completion services

You can use the chat completion service directly, without adding to the kernel.

In [7]:
# Instantiatte execution settings
execution_settings = OpenAIChatPromptExecutionSettings()

### a) Non-streaming chat completion

In [8]:
# Initialise new chat history
chat_history = ChatHistory()
chat_history.add_user_message("Can you prepare a short (3 paragraphs) story about red pandas?")

In [9]:
# Get non-streaming response
response = await chat_completion_service.get_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

print(response)

In the misty forests of the Eastern Himalayas, a red panda named Pema awoke to the soft rustling of bamboo leaves. With her thick, reddish-brown fur blending seamlessly into the autumn foliage, she nimbly climbed from one branch to another in search of her favorite treat—bamboo shoots. Though small and elusive, red pandas like Pema are expert climbers, using their sharp claws and long, bushy tails for balance as they navigate the treetops.

Pema's days were often spent foraging, but she was also incredibly curious. One afternoon, while exploring a new part of the forest, she spotted a group of colorful birds fluttering near a sparkling stream. Intrigued, Pema carefully made her way down to the water’s edge, where she paused to drink and watch the lively scene. Despite her solitary nature, moments like these connected her to the vibrant ecosystem around her—where every creature, big or small, played a vital role.

As twilight fell, Pema found a cozy nook nestled among the branches to re

### b) Streaming chat completion

In [10]:
# Initialise new chat history
chat_history = ChatHistory()
chat_history.add_user_message("Can you prepare a short (3 paragraphs) story about red pandas?")

In [11]:
# Get streaming response
response = chat_completion_service.get_streaming_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

async for chunk in response:
    print(chunk, end="")

Deep within the misty forests of the Eastern Himalayas, a tiny red panda named Kiri scurried through the dense bamboo groves. With her bright reddish fur blending seamlessly with the autumn leaves, Kiri was a master of camouflage. Despite her small size, she was agile and curious, always exploring the treetops where she felt safest among the swaying branches. Each day, Kiri would nibble on fresh bamboo shoots, her favorite treat, while occasionally pausing to observe the colorful birds and buzzing insects that shared her home.

One crisp morning, Kiri discovered a shimmering stream she had never seen before. Intrigued by the sparkling water, she delicately stepped closer, balancing on a narrow log. As she gazed at her reflection, a sudden rustling in the bushes startled her. Out hopped a young snow leopard cub, equally curious and wary. Instead of fleeing, Kiri stood her ground, and the two animals shared a silent moment of understanding, bound by their shared wilderness. Together, the

### 🪜 Step 5: Use rich messages in chat history

In [12]:
# Public image URL for demonstration
hotel_room_image_url = "https://images.pexels.com/photos/164595/pexels-photo-164595.jpeg"

print(f"Anya Sharma's reference image for a hotel room:")
display(Image(url=hotel_room_image_url, width=400)) # Display the image directly in the notebook

Anya Sharma's reference image for a hotel room:


In [13]:
# Initialise new chat history
chat_history = ChatHistory()

chat_history.add_message(
    ChatMessageContent(
        role = AuthorRole.SYSTEM,
        content = "You are a helpful hotel booking assistant. You can assist users in finding and booking rooms, answer questions about hotel amenities, and consider their preferences. Provide personalised suggestions where possible."
    )
)

chat_history.add_message(
    ChatMessageContent(
        role = AuthorRole.USER,
        name = "Anya_Sharma",
        items = [
            TextContent(text="I'm looking to book a room. Can you help me find something similar to what's in this picture?"),
            ImageContent(uri=hotel_room_image_url) # Referencing the image displayed previously
        ]
    )
)

In [14]:
# Simulate function call and result (Tool Message) for user's preferences
chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.ASSISTANT,
        items=[
            FunctionCallContent(
                name = "get_user_preferences_HotelBookingPlugin",
                id = "user_prefs_anya_001",
                arguments = str({"username": "Anya_Sharma"})
            )
        ]
    )
)

# Tool provides the result of the function call (simulated)
chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.TOOL,
        items=[
            FunctionResultContent(
                name = "get_user_preferences_HotelBookingPlugin",
                id = "user_prefs_anya_001",
                result = '{ "loyalty_status": "Diamond", "preferred_amenities": ["king_bed", "balcony", "high_floor", "late_checkout"] }'
            )
        ]
    )
)

In [15]:
# Get assistant's response
assistant_response = await chat_completion_service.get_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

chat_history.add_message(assistant_response)
print(f"Assistant: {assistant_response.content}")

Assistant: I see you are looking for a room similar to the one in the picture, which features a king-sized bed, a modern and cozy design with neutral tones and artistic decor. Considering your preferences for a king bed, balcony, high floor, and late checkout, I can help you find a room that matches this style and your requirements.

Could you please tell me the location and dates for your stay? This will help me find the best options for you.


In [16]:
# Display the full chat history
print("\n--- Full Rich Chat History ---")
for message in chat_history:
    print(
        f"Role: {message.role}",
        f"Name: {message.name}",
        f"Content: {message.content}",
        f"Items: {message.items}\n"
    )


--- Full Rich Chat History ---
Role: AuthorRole.SYSTEM Name: None Content: You are a helpful hotel booking assistant. You can assist users in finding and booking rooms, answer questions about hotel amenities, and consider their preferences. Provide personalised suggestions where possible. Items: [TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text='You are a helpful hotel booking assistant. You can assist users in finding and booking rooms, answer questions about hotel amenities, and consider their preferences. Provide personalised suggestions where possible.', encoding=None)]

Role: AuthorRole.USER Name: Anya_Sharma Content: I'm looking to book a room. Can you help me find something similar to what's in this picture? Items: [TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text="I'm looking to book a room. Can you help me find something similar to what's in this picture?", encoding=None), ImageContent(inner_conte